<a href="https://colab.research.google.com/github/HanselWilfred/WeatherWise-Hansel-Wilfred/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install requests matplotlib pyinputplus







## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather

# Add any other setup code here

## 🌤️ Weather Data Functions

In [ ]:
# ==== WeatherWise: ====
import requests, re
def get_weather_data(loc, forecast_days=5):
    try:
        days = max(1, min(5, int(forecast_days or 5)))
        loc = re.sub(r"\b(tomorrow|today|day after|in\s+\d+\s+days)\b", "", str(loc), flags=re.I)
        loc = re.sub(r"\s+", " ", loc).strip()
        if not loc: return {}
        g = requests.get(
            "https://geocoding-api.open-meteo.com/v1/search",
            params={"name": loc, "count":1, "language": "en"},
            timeout=10
        )
        g.raise_for_status()
        hits = (g.json().get("results") or [])
        if not hits: return {}
        lat, lon = hits[0]["latitude"], hits[0]["longitude"]
        city = hits[0].get("name", loc)
        f = requests.get(
            "https://api.open-meteo.com/v1/forecast",
            params={"latitude": lat, "longitude": lon, "timezone": "auto",
                    "current_weather": True,
                    "daily": "temperature_2m_max,temperature_2m_min,precipitation_probability_max"},
            timeout=10
        )
        f.raise_for_status()
        d = f.json().get("daily", {})
        dates, tmax, tmin, rain = d.get("time", []), d.get("temperature_2m_max", []), d.get("temperature_2m_min", []), d.get("precipitation_probability_max", [])
        n = min(days, len(dates))
        fc = [{"date": dates[i], "max_temp": tmax[i], "min_temp": tmin[i], "rain_chance": rain[i]} for i in range(n)]
        return {"location": city, "current_temp": f.json().get("current_weather", {}).get("temperature"), "forecast": fc}
    except Exception:
        return {}





## 📊 Visualisation Functions

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from datetime import datetime

def matplotlib_3d_temperature(weather_data):
    if not weather_data or "error" in weather_data or not weather_data.get("forecast"):
        print("No data to visualise."); return
    fc = weather_data['forecast']
    dates = [datetime.strptime(d['date'], "%Y-%m-%d") for d in fc]
    tmin = [d['min_temp'] for d in fc]
    tmax = [d['max_temp'] for d in fc]

    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')

    # Assign numeric values for date axis
    date_nums = np.arange(len(dates))
    ax.plot(date_nums, tmin, zs=0, zdir='y', label='Min Temp', color='blue', marker='o')
    ax.plot(date_nums, tmax, zs=1, zdir='y', label='Max Temp', color='red', marker='o')

    # Set axes and labels
    ax.set_xlabel('Date')
    ax.set_ylabel('Temp Type')
    ax.set_zlabel('Temperature (°C)')
    ax.set_xticks(date_nums)
    ax.set_xticklabels([d.strftime("%d %b") for d in dates])
    ax.set_yticks([0,1])
    ax.set_yticklabels(['Min Temp', 'Max Temp'])
    ax.set_title(f"3D Min/Max Temperature — {weather_data.get('location','Location')}")
    ax.legend()
    plt.show()






In [ ]:
# 2

def matplotlib_bar_rain(weather_data):
    if not weather_data or "error" in weather_data or not weather_data.get("forecast"):
        print("No data to visualise."); return
    import matplotlib.pyplot as plt
    from datetime import datetime

    fc = weather_data['forecast']
    rain = [float(d.get('rain_chance', 0)) for d in fc]
    dates = [datetime.strptime(d['date'], "%Y-%m-%d").strftime("%d %b") for d in fc]

    fig, ax = plt.subplots(figsize=(1.3 * len(dates), 5))
    bars = ax.bar(dates, rain, color='skyblue', edgecolor='navy')
    for rect, val in zip(bars, rain):
        ax.text(rect.get_x() + rect.get_width()/2, val + 2,
                f"{int(val)}%", ha='center', va='bottom', fontsize=10)
    ax.set_ylabel('Chance of Rain (%)')
    ax.set_ylim(0, 100)
    ax.set_title(f"Rain Chance per Day — {weather_data.get('location','Location')}")
    plt.tight_layout()
    plt.show()





## 🤖 Natural Language Processing

In [ ]:
# === Section 2: NLP + Response (with formatted dates) ===
import re
from datetime import datetime

def parse_weather_question(q):
    q = (q or "").lower().strip()
    day = 2 if ("day after tomorrow" in q or "day after" in q) else 1 if "tomorrow" in q else 0
    m = re.search(r"\bin\s+(\d+)\s+day(s)?\b", q)
    if m:
        try: day = max(0, min(4, int(m.group(1))))
        except: pass
    attr = "rain" if any(w in q for w in ["rain", "rainy", "umbrella", "drizzle", "shower"]) else "temperature" if any(w in q for w in ["temp", "hot", "cold", "warm", "cool", "chill"]) else "general"
    pin = re.search(r"\b\d{3,6}\b", q)
    if pin: return {"location": pin.group(0), "day": day, "attribute": attr}
    m = re.search(r"\bin\s+([a-z0-9\s\-]+?)(?=(?:\?|\.|,|$|\s+tomorrow|\s+today|\s+day after|\s+in\s+\d+\s+day[s]?))", q)
    loc = None
    if m:
        loc = re.sub(r"\b(tomorrow|today|day after tomorrow|day after|in\s+\d+\s+day[s]?)\b", "", m.group(1), flags=re.I).strip().title()
        if not loc: loc = None
    return {"location": loc, "day": day, "attribute": attr}

def generate_weather_response(parsed_question, data):
    loc, day, attr = parsed_question.get("location"), parsed_question.get("day",0), parsed_question.get("attribute","general")
    if not loc: return "Please tell me a city or PIN/ZIP (e.g., 'weather in Perth')."
    if not data or "error" in data: return data.get("error", "Sorry, I couldn't get the weather right now.")
    fc = data.get("forecast") or []
    if not fc: return "No forecast available right now."
    i = min(day, len(fc)-1)
    d = fc[i]
    city = data.get("location", loc)
    date = d.get("date", "that day")
    try: date = datetime.strptime(date, "%Y-%m-%d").strftime("%d %b %Y")
    except: pass
    mx, mn = d.get("max_temp"), d.get("min_temp")
    rc = d.get("rain_chance")
    t = mx if mx is not None else mn
    def temp_phrase(x):
        if x is None: return "temperature unavailable"
        return "hot and sunny" if x >= 35 else "cool and pleasant" if x >= 15 else "cold"
    def rain_phrase(v):
        if v is None: return "No rain info."
        if v >= 50: return f"Yes, likely — rain {int(v)}%."
        if v >= 20: return f"Possible — rain {int(v)}%."
        return f"Unlikely — rain {int(v)}%."
    if attr == "rain": return f"In {city} on {date}: {rain_phrase(rc)}"
    msg = f"In {city} on {date}: {temp_phrase(t)}"
    if t is not None: msg += f" (around {round(t)}°C)"
    if rc is not None and rc >=30: msg += f", rain {int(rc)}%."
    else: msg += "."
    return msg


## 🧭 User Interface

In [ ]:
# === UI (A):

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

def print_forecast(res):
    if not res or "error" in res:
        print(f"⚠️ {res.get('error','No data')}")
        return
    cur = res.get('current_temp')
    print(f"📍 {res['location']} | 🌡️ {cur}°C" if isinstance(cur, (int,float)) else "N/A")
    for f in res.get("forecast", []):
        d = f.get('date','')
        try: d = datetime.strptime(d, "%Y-%m-%d").strftime("%d %b")
        except: pass
        temp = f.get("min_temp") if isinstance(f.get("min_temp"), (int,float)) else f.get("max_temp") if isinstance(f.get("max_temp"), (int,float)) else None
        emoji = "🔥" if temp and temp >= 35 else "🙂" if temp and temp >= 15 else "🥶"
        temp_str = f"{f.get('min_temp'):.1f}–{f.get('max_temp'):.1f}°C" if all(isinstance(t, (int,float)) for t in [f.get("min_temp"), f.get("max_temp")]) else "temp N/A"
        rc = f.get("rain_chance")
        try: rcv = float(rc)
        except: rcv = None
        rain_note = f" (rain {int(rcv)}%)" if rcv and rcv >=30 else ""
        print(f"{d}: {temp_str} {emoji}{rain_note}")

def create_temperature_visualisation(res):
    if not res or "error" in res or not res.get("forecast"):
        print("No data")
        return
    fc = res["forecast"]
    dates = [datetime.strptime(r["date"], "%Y-%m-%d") for r in fc]
    tmin = [r.get("min_temp") for r in fc]
    tmax = [r.get("max_temp") for r in fc]

    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection="3d")

    date_nums = np.arange(len(dates))
    ax.plot(date_nums, [0]*len(dates), tmin, color="blue", marker="o", label="Min Temp")
    ax.plot(date_nums, [1]*len(dates), tmax, color="red", marker="o", label="Max Temp")

    ax.set_xlabel("Date")
    ax.set_ylabel("Temp Type")
    ax.set_zlabel("Temperature (°C)")
    ax.set_xticks(date_nums)
    ax.set_xticklabels([d.strftime("%d %b") for d in dates])
    ax.set_yticks([0,1])
    ax.set_yticklabels(["Min", "Max"])
    ax.set_title(f"3D Min/Max Temp — {res.get('location','Location')}")
    ax.legend()
    plt.show()



In [ ]:
# === UI (B):

def create_precipitation_visualisation(res):
    if not res or "error" in res or not res.get("forecast"):
        print("No data")
        return
    fc = res["forecast"]
    xs = range(len(fc))
    dates = [datetime.strptime(r["date"], "%Y-%m-%d").strftime("%d %b") for r in fc]
    rainp = [max(0, min(100, float(r.get("rain_chance",0)))) for r in fc]
    plt.fill_between(xs, 0, rainp, alpha=0.15)
    plt.plot(xs, rainp, linewidth=2)
    for x, v in zip(xs, rainp):
        plt.vlines(x, 0, v)
        plt.scatter(x, v, s=50)
        plt.text(x, v+2, f"{int(v)}%", ha="center", fontsize=8)
    plt.title(f"Rain Chance — {res.get('location')}")
    plt.ylim(0, 100)
    plt.xticks(xs, dates)
    plt.tight_layout()
    plt.show()

def create_both_visualisations(res):
    create_temperature_visualisation(res)
    create_precipitation_visualisation(res)




## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def run_app():
    print("🌦️ Welcome to WeatherWise 🌦️")
    print("Enter city name or PIN/ZIP.")
    while True:
        print("\nOptions:\n1. View Forecast\n2. Ask Question\n3. Visualise Data\n4. Exit")
        c = input("> ").strip()
        if c == "1":
            p = input("City or PIN/ZIP: ").strip()
            print_forecast(get_weather_data(p))
        elif c == "2":
            q = input("Ask about weather: ").strip()
            info = parse_weather_question(q)
            p = info.get("location")
            if not p:
                p = input("City or PIN/ZIP: ").strip()
                q = f"{q} in {p}"
            print(generate_weather_response(q, get_weather_data))
        elif c == "3":
            p = input("City to visualise: ").strip()
            res = get_weather_data(p)
            if not res or "error" in res:
                print(f"⚠️ {res.get('error','No data')}")
                continue

            print("1) Temperature  2) Rain Chance 3) Both")
            v = input("> ").strip().lower()

            if v in ("1", "temp", "temperature"):
                create_temperature_visualisation(res)
            elif v in ("2", "rain", "precip", "precipitation", "violin"):
                matplotlib_violin_rain(res)
            elif v in ("3", "both", "all"):
                create_temperature_visualisation(res)
                matplotlib_violin_rain(res)
            else:
                print("Choose 1, 2, or 3.")
        elif c == "4":
            print("Goodbye! ☀️")
            break
        else:
            print("Choose 1–4.")



## 🧪 Testing and Examples

In [ ]:
4
# Questions you can ask weatherwise
SAMPLE_QUESTIONS = [
    # Rain-only
    "Will it rain today?",
    "Is rain likely in Chennai tomorrow?",

    # Temperature-only
    "Is it cold tomorrow in Melbourne?",
    "Temperature in Adelaide?",
    "Tell me the weather in 6000?",

    # General feel
    "How is the weather?",
    "Show me the temperature in Brisbane in 2 days?",
    "Do I need an umbrella tomorrow in Perth?",

]

run_app()